In [1]:
import time
import bs4 #Utilisation de beautifulsoup pour le scraping
import pandas
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from urllib import request    

In [2]:
list_questions = []
list_votes = []
list_answers = []
list_views = []

In [3]:
for i in range (1):
    time.sleep(1)
    url = "https://stackoverflow.com/questions?tab=newest&page="+str(i)
    request_text = request.urlopen(url).read()
    page = bs4.BeautifulSoup(request_text,'html.parser')
    questions = page.select(".s-post-summary")

In [4]:
      for question in questions:
            time.sleep(1)
            q = question.select_one('.s-link').get_text() # on récupère les titres de questions
            stats = question.select('.s-post-summary--stats')
            list_questions.append(q)
            for sta in stats:
                votes = sta.select('[title~=Score]')
                answers = sta.select('[title~=answer],[title~=answers]')
                views = sta.select('[title~=views]')
                for vote in votes:
                    votes_number = vote.select_one('.s-post-summary--stats-item-number').get_text() # on récupère le nombre de votes
                    votes_text= vote.select_one('.s-post-summary--stats-item-unit').get_text()
                    list_votes.append(votes_number)
                for answer in answers:
                    answers_number = answer.select_one('.s-post-summary--stats-item-number').get_text() # on récupère le nombre de réponses
                    answers_text = answer.select_one('.s-post-summary--stats-item-unit').get_text()
                    list_answers.append(answers_number)
                for view in views:
                    views_number = view.select_one('.s-post-summary--stats-item-number').get_text() # on récupère le nombre de vues
                    views_text = view.select_one('.s-post-summary--stats-item-unit').get_text()
                    list_views.append(views_number)


In [5]:
df = pandas.DataFrame.from_dict( {"Questions" : list_questions, "Votes" : list_votes, "Answers" : list_answers, "Vues" : list_views})
df

,Questions,Votes,Answers,Vues
0,Why sublist access time increases with sublist...,0,0,3
1,Fetching SAP-BW reports via an API,0,0,2
2,cost header missing for delta query on group,0,0,2
3,"""The irony is not lost on me"" meaning in this ...",0,0,3
4,Accessing nested XML - going upwards,0,0,2
5,getFloat isn't there in java to get Float data...,0,0,2
6,Filtering react array of objects is not workin...,0,0,3
7,Tensorflow GPU install test failing (PopOS),0,0,2
8,SOLR and garbage collection,0,0,2
9,jeopardy api clue issues,0,0,2
